In [6]:
import pandas as pd
import numpy as np
from matplotlib_venn import venn2
import matplotlib.pyplot as plt

In [ ]:
# df = pd.read_csv('/rhome/slokr001/shared/slokray/fgsea_reac_results_edgeR_BH.csv')
df = pd.read_csv('/rhome/slokr001/shared/slokray/fgsea_kegg_results_edgeR_BH.csv')
# df = pd.read_csv('/rhome/slokr001/shared/slokray/fgsea_go_results edgeR_BH.csv')

print('loaded csv')

df_interleaved = pd.DataFrame()

for comparison in df['Comparison'].unique():
    temp_df = df[df['Comparison'] == comparison][['pathway', 'padj', 'NES']].copy()
    temp_df.columns = ['pathway', f'padj_{comparison}', f'NES_{comparison}']

    if df_interleaved.empty:
        df_interleaved = temp_df
    else:
        df_interleaved = pd.merge(df_interleaved, temp_df, on='pathway', how='outer')

print('sorted by comparison')

columns = df_interleaved.columns
padj_columns = list(columns[1::2])

for idx, row in df_interleaved.iterrows():
    drop = True
    padj_values = list(row[padj_columns])
    for val in padj_values:
        if val < 0.05:
            drop = False
            break
    if drop:
        df_interleaved.drop(idx, inplace=True)

sorted_pathways = df_interleaved['pathway'].tolist()

print('sorted pathways')

filtered_pathways = []
for i in sorted_pathways:
  tokens = i.split(' - ')
  filtered_pathways.append(tokens[-1].upper())

sig_pathways = pd.read_csv('/content/sig_pathways.csv')
sig_filtered_pathways = []
for i in sig_pathways['GSEA Pathway']:
  tokens = i.split('KEGG ')
  if len(tokens) > 1:
    sig_filtered_pathways.append(tokens[-1])

loaded csv


In [7]:
import dask.dataframe as dd

# Load the CSV with Dask
df = dd.read_csv('/rhome/slokr001/shared/slokray/fgsea_kegg_results_edgeR_BH.csv')

# Initialize an empty DataFrame for interleaved results
df_interleaved = dd.from_pandas(pd.DataFrame(), npartitions=1)

# Iterate over unique comparisons and merge results
for comparison in df['Comparison'].unique().compute():  # Compute to get unique values
    temp_df = df[df['Comparison'] == comparison][['pathway', 'padj', 'NES']].copy()
    temp_df.columns = ['pathway', f'padj_{comparison}', f'NES_{comparison}']
    
    # Merge on 'pathway' using outer join
    df_interleaved = df_interleaved.merge(temp_df, on='pathway', how='outer')

print('Sorted by comparison')

# Identify adjusted p-value columns
padj_columns = df_interleaved.columns[1::2].tolist()

# Filter rows where any padj values are less than 0.05
df_interleaved = df_interleaved[(df_interleaved[padj_columns] < 0.05).any(axis=1)]

sorted_pathways = df_interleaved['pathway'].compute().tolist()  # Compute to get list

print('Sorted pathways')

# Process pathways to filter and format names
filtered_pathways = [i.split(' - ')[-1].upper() for i in sorted_pathways]

# Load significant pathways and process them
sig_pathways = pd.read_csv('/content/sig_pathways.csv')
sig_filtered_pathways = [i.split('KEGG ')[-1] for i in sig_pathways['GSEA Pathway'] if 'KEGG ' in i]

# Display final results (if needed)
print("Filtered Pathways:", filtered_pathways)
print("Significant Filtered Pathways:", sig_filtered_pathways)


KeyError: 'pathway'

In [ ]:
venn2([set(filtered_pathways), set(sig_filtered_pathways)], set_labels=('Analysis', 'Paper'))
plt.title('KEGG edgeR')
plt.show()